# Предобработка данных для обучения

### Загрузка данных и первоначальная обработка

In [ ]:
import os
import pandas as pd
import pyarrow.parquet as pq
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras import layers, models, initializers, callbacks
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score
from imblearn.over_sampling import ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
import warnings
import joblib
import numpy as np

warnings.filterwarnings('ignore', category=FutureWarning)

def amountOfNone(df, text):
    rows_with_missing_values = df.isna().any(axis=1)
    count_rows_with_missing_values = rows_with_missing_values.sum()
    print(f"{text}: {count_rows_with_missing_values}")

print("Идет удаление...")
df = pq.read_table("train_ai_comp_final_dp.parquet").to_pandas()
df.drop(['id','sample_ml_new', 'feature642', 'feature756'], axis=1, inplace=True)

### Нормализация данных

In [ ]:
print("Нормализация...")
numeric_columns = df.select_dtypes(include=['number']).columns.drop('target')
# Создаем экземпляр класса MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
# Применяем MinMaxScaler к выбранным числовым столбцам
df[numeric_columns] = scaler.fit_transform(df[numeric_columns])

X = df.drop(['target'], axis=1)
y = df['target']

### Сэмплирование данных

In [ ]:
# Создаем пайплайн для ADASYN и RandomUnderSampler
pipeline = Pipeline([
    ('adasyn', ADASYN(sampling_strategy={1: 30000}, n_neighbors=5, random_state=42)),
    ('under', RandomUnderSampler(sampling_strategy={0: 30000}, random_state=42))
])

X_resampled, y_resampled = pipeline.fit_resample(X, y)

df_resampled = pd.concat([pd.DataFrame(y_resampled), pd.DataFrame(X_resampled, columns=X.columns)], axis=1)
df_resampled.drop_duplicates(inplace=True)
print(f"После ресемплинга количество экземпляров 'target=1': {(df_resampled['target'] == 1).sum()}")
print(f"После ресемплинга количество экземпляров 'target=0': {(df_resampled['target'] == 0).sum()}")

amountOfNone(df_resampled, "Количество строк с пропущенными значениями после ресемплинга")

# Обучение модели логистической регрессии

In [ ]:
# Предположим, что target_column - это название вашей целевой колонки, а остальные колонки - это признаки
X = df_resampled.drop('target', axis=1)
y = df_resampled['target']

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Создание и обучение модели
logistic_model = LogisticRegression(penalty="l2",solver='saga',class_weight='balanced',tol=1e-7,max_iter=10000, random_state=42)
logistic_model.fit(X_train, y_train)


# Прогнозирование вероятностей для обучающего и тестового наборов
y_train_pred = logistic_model.predict_proba(X_train)[:, 1]
y_test_pred = logistic_model.predict_proba(X_test)[:, 1]

# Расчет метрик
train_logloss = log_loss(y_train, y_train_pred)
train_auc = roc_auc_score(y_train, y_train_pred)
test_logloss = log_loss(y_test, y_test_pred)
test_auc = roc_auc_score(y_test, y_test_pred)

print(f"Train Logloss: {train_logloss}, Train AUC: {train_auc}")
print(f"Test Logloss: {test_logloss}, Test AUC: {test_auc}")

# Сохранение модели
joblib.dump(logistic_model, 'logistic_regression_model.pkl')

### Проверка модели логической регрессии

In [ ]:

def load_model(path_to_model):
    # Загрузка модели из файла
    return joblib.load(path_to_model)

def predict(model, data):
    # Получение предсказаний от модели
    predictions = model.predict_proba(data)[:, 1]
    return predictions

def load_data(path_to_data):
    # Загрузка данных
    df = pq.read_table(path_to_data).to_pandas()
    features = ['sample_ml_new','feature756','feature642',]

    # Предположим, что 'target' - это название вашей целевой колонки, которую мы удаляем для предсказания
    X = df.drop(features,axis=1)

    numeric_columns = X.select_dtypes(include=['number']).columns.drop('id')
    # Создаем экземпляр класса MinMaxScaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    # Применяем MinMaxScaler к выбранным числовым столбцам
    X[numeric_columns] = scaler.fit_transform(X[numeric_columns])

    return X

path_to_model = 'logistic_regression_model.pkl'
path_to_data = 'test_sber.parquet'

# Загрузка модели
model = load_model(path_to_model)

# Загрузка данных
data = load_data(path_to_data)
data2 = data.drop('id',axis=1)
# Получение и вывод предсказаний
predictions = predict(model,data2)
prediction_rounded = np.round(predictions).astype(int)
df1 = pd.DataFrame({
    "id": data['id'],
    "target_bin": prediction_rounded,
    "target_prob": predictions
})

df1.to_csv('predicions_test_logic.csv',index=False)